In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import RSRAE

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
        config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
detector = RSRAE(
    encoder_dims=[500, 450, 400, 350, 300],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    rsr_dim=250,
    batch_norm=True,
)

In [4]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [5]:
# Training
epochs = 10
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-05-22 17:29:10,868 [INFO] __main__: Start training the model...
2024-05-22 17:29:10,868 [INFO] __main__: Epoch: 1
2024-05-22 17:29:11,612 [INFO] __main__: Loss: 533.092407
2024-05-22 17:29:12,083 [INFO] __main__: Loss: 188.272400
2024-05-22 17:29:12,545 [INFO] __main__: Loss: 106.492523
2024-05-22 17:29:13,052 [INFO] __main__: Loss: 77.755112
2024-05-22 17:29:13,532 [INFO] __main__: Loss: 65.129623
2024-05-22 17:29:13,624 [INFO] __main__: Epoch: 2
2024-05-22 17:29:13,630 [INFO] __main__: Loss: 64.354080
2024-05-22 17:29:14,110 [INFO] __main__: Loss: 57.979958
2024-05-22 17:29:14,565 [INFO] __main__: Loss: 48.900391
2024-05-22 17:29:15,069 [INFO] __main__: Loss: 45.762047
2024-05-22 17:29:15,532 [INFO] __main__: Loss: 43.174583
2024-05-22 17:29:15,572 [INFO] __main__: Epoch: 3
2024-05-22 17:29:15,578 [INFO] __main__: Loss: 42.712120
2024-05-22 17:29:16,067 [INFO] __main__: Loss: 41.275291
2024-05-22 17:29:16,555 [INFO] __main__: Loss: 38.750240
2024-05-22 17:29:17,080 [INFO] __main

In [6]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-05-22 17:29:31,673 [INFO] __main__: Start evaluating the model...
2024-05-22 17:29:32,339 [INFO] __main__: AUROC Score: 0.4324809299243133
2024-05-22 17:29:32,340 [INFO] __main__: Completed the evaluation of the model!
